In [1]:
import sys, os
import pandas as pd
import setting

import sys, os
import pandas as pd
import setting
import pathSetting
sys.path.append("../../..")

from Clust.clust.ML.common.common import p1_integratedDataSaving as p1
from Clust.clust.ML.common.common import p2_dataSelection as p2
from Clust.clust.ML.common.common import p3_training as p3
from Clust.clust.ML.common import model_manager

import torch

#import main_regression as mr
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cpu is available.


# 2. Training 

## 2-1. Data selection

In [2]:
dataRoot = 'ml_data_integration'
# 1 (p2부분 1. Data Selection)
DataMeta = p1.read_json_data(pathSetting.DataMetaPath)
dataList =  list(DataMeta.keys())

In [3]:
dataList

['IntegraionTrainX', 'IntegraionTrainy', 'IntegraionTestX', 'IntegraionTesty']

In [4]:
#2
# dataX
dataName_X = dataList[0]
dataSaveMode_X = DataMeta[dataName_X]["integrationInfo"]["DataSaveMode"]

# datay
dataName_y = dataList[1]
dataSaveMode_y = DataMeta[dataName_y]["integrationInfo"]["DataSaveMode"]

#3
dataFolderName = "data_integrated_result"
current = os.getcwd()
dataFolderPath = os.path.join(current, dataFolderName)
dataX = p2.get_saved_integrated_data(dataSaveMode_X, dataName_X, dataFolderPath)
datay = p2.get_saved_integrated_data(dataSaveMode_y, dataName_y, dataFolderPath)
integration_freq_sec = DataMeta[dataName_X]["integrationInfo"]["integration_freq_sec"]

## 2-2. Training Data Preparation

In [5]:
# 2 Training Data Preparation
# 2-1
featureListX= list(dataX.columns)
featureListy= list(datay.columns)

# 2-2
cleanTrainDataParam = 'NoClean'#  Classification, Regression과 같이 X, y가 분리된 경우에는 현재 고정해서 사용해야함

# 2-2-1 cleanTrainDataParam == Clean 일 경우
NaNProcessingParam ={
    "feature_cycle":'Day',
    "feature_cycle_times":1,
    "NanInfoForCleanData":{'type':'num', 'ConsecutiveNanLimit':3, 'totalNaNLimit':30000}
}
# 2-3
scalerParam='scale'
scaleMethod='minmax'

# 2-4
splitRatio = 0.8

# 2-5
scalerRootPath_X = os.path.join(pathSetting.scalerRootDir, dataName_X, cleanTrainDataParam)
scalerRootPath_y = os.path.join(pathSetting.scalerRootDir, dataName_X, cleanTrainDataParam)
train_x, val_x, X_scalerFilePath = p3.get_train_val_data(dataX, featureListX, scalerRootPath_X, splitRatio, scalerParam, scaleMethod)
train_y, val_y, y_scalerFilePath = p3.get_train_val_data(datay, featureListy, scalerRootPath_y, splitRatio, scalerParam, scaleMethod)

['RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7', 'RH_8', 'RH_9', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'Press_mm_hg', 'RH_out', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
Make New scaler File
['value']
Make New scaler File


## 2-3 Training 

In [6]:
# 3-1.
model_list = ["LSTM_rg","CNN_1D_rg","GRU_rg", "LSTM_FCNs_rg"]
model_method = model_list[0]

#import main_regression as mr

n_epochs = 10 # 학습 epoch 횟수, int(default: 1000, 범위: 1 이상)
batch_size = 16  # batch 크기, int(default: 16, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)

trainParameter = setting.modelTestconfig[model_method]
trainParameter['device']  = device
trainParameter['n_epochs'] = n_epochs
trainParameter['batch_size'] = batch_size

modelTags =["aaaaa"]
trainDataType = "timeseries"
modelPurpose = "regression"

# 2
trainDataInfo = DataMeta[dataName_X]['integrationInfo']

# param = {
#     "clean_param":cleanMode,
#     "transform_parameter":transformParameter,
#     "batch_size":batch_size,
#     "model_parameter":trainParameter,
#     "n_epochs":n_epochs,
#     "n_epochs":n_epochs
# }

# 3. 모델을 저장할 파일 패스를 생성한다.

model_name = 'EnergyLSTMRegression'
from Clust.clust.transformation.general.dataScaler import encode_hash_style
trainParameter_encode =  encode_hash_style(str(trainParameter))
# trainDataPathList = ["CLUST","Electronics", dataName_X, trainParameter_encode]
trainDataPathList = [model_name, dataName_X, trainParameter_encode]
modelFilePath = model_manager.get_model_file_path(trainDataPathList, model_method)

# 4. Training
from Clust.clust.ML.regression.train import RegressionML as RML
rml = RML()
rml.set_param(trainParameter)
rml.set_data(train_x, train_y, val_x, val_y)
rml.set_model(model_method)
model = rml.train()

./Models/LSTM_rg/EnergyLSTMRegression/IntegraionTrainX/618f2ca7c380b25efe12bee2a2ab1869/model.pkl
{'num_layers': 2, 'hidden_size': 64, 'dropout': 0.1, 'bidirectional': True, 'lr': 0.0001, 'device': 'cpu', 'n_epochs': 10, 'batch_size': 16}
Start training model

Epoch 1/10


../../../Clust/clust/transformation/type/DFToNPArray.py:38: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  dfX_partial = dfX[startDate:endDate]
../../../Clust/clust/transformation/type/DFToNPArray.py:39: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  dfy_partial = dfy[startDate:endDate]
../../../Clust/clust/transformation/type/DFToNPArray.py:38: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  dfX_partial = dfX[startDate:endDate]
../../../Clust/clust/transformation/type/DFToNPArray.py:39: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecat

train Loss: 0.2104
val Loss: 0.1532

Epoch 10/10
train Loss: 0.0747
val Loss: 0.0405

Training complete in 0m 6s
Best val MSE: 0.040545


In [ ]:
from Clust.clust.ML.tool import model as model_manager
model_manager.save_pickle_model(model)

In [7]:
from Clust.clust.ingestion.mongo import mongoClient as mongo_client
from Clust.setting import influx_setting_KETI as ins

mongo_client = mongo_client.mongoClient(ins.CLUSTMetaInfo2)

In [8]:
# 5. update MetaFile
# ModelName = encode_hash_style(p1.get_list_merge ([model_method, cleanTrainDataParam, trainParameter_encode, dataName_X, dataName_y] ))
# model_name = ''
modelInfoMeta ={
    "author":"KETI",
    "modelName": model_name,
    "modelVersion":0.1,
    "trainDataInfo":trainDataInfo,
    "featureList":featureListX,
    "target":featureListy,
    "trainDataType":trainDataType,
    "modelPurpose":modelPurpose,
    "model_method":model_method,
    "modelTags":modelTags,
    "cleanTrainDataParam":cleanTrainDataParam,
    "NaNProcessingParam":NaNProcessingParam,
    "trainDataName":[dataName_X,dataName_y],
    "trainParameter":rml.parameter,
    "scalerParam":scalerParam,
    "files":{
        "modelFile":{
            "fileName":"model.pth",
            "filePath":modelFilePath
        },
        "XScalerFile":{
            "fileName":"scaler.pkl",
            "filePath":X_scalerFilePath       
        },
        "yScalerFile":{
            "fileName":"scaler.pkl",
            "filePath":y_scalerFilePath       
        }
    },
    "trainParameter":trainParameter,
    "n_epochs":n_epochs
}
# modelInfoMeta = p3.updateModelMetaData(ModelName, modelInfoMeta, pathSetting.trainModelMetaFilePath)
modelInfoMeta = model_manager.update_model_meta_data(mongo_client, modelInfoMeta)

Success
{'author': 'KETI', 'modelName': 'EnergyLSTMRegression', 'modelVersion': 0.1, 'trainDataInfo': {'startTime': '2016-01-11', 'endTime': '2016-04-15', 'dataInfo': [['life_indoor_environment', 'humidityTrain_10min'], ['life_indoor_environment', 'temperatureTrain_10min'], ['weather_outdoor_environment', 'belgiumChieverseAirportTrain_10min']], 'processParam': {'refine_param': {'removeDuplication': {'flag': False}, 'staticFrequency': {'flag': False, 'frequency': None}}, 'outlier_param': {'certainErrorToNaN': {'flag': False}, 'unCertainErrorToNaN': {'flag': False, 'param': {}}}, 'imputation_param': {'flag': False, 'imputation_method': [], 'totalNonNanRatio': 80}}, 'integration_freq_sec': 600, 'cleanParam': 'NoClean', 'DataSaveMode': 'CSV'}, 'featureList': ['RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7', 'RH_8', 'RH_9', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'Press_mm_hg', 'RH_out', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed'], 'target': ['value'], 'trainDataTy

In [9]:
from Clust.clust.ingestion.mongo import mongoClient as mongo_client
from Clust.setting import influx_setting_KETI as ins

mongo_client = mongo_client.mongoClient(ins.CLUSTMetaInfo2)

db_name = 'model'
collection_name = 'meta'

all_model_meta = mongo_client.get_all_document(db_name, collection_name)
all_model_meta

[{'author': 'KETI',
  'modelName': 'EnergyLSTMRegression',
  'modelVersion': 0.1,
  'trainDataInfo': {'startTime': '2016-01-11',
   'endTime': '2016-04-15',
   'dataInfo': [['life_indoor_environment', 'humidityTrain_10min'],
    ['life_indoor_environment', 'temperatureTrain_10min'],
    ['weather_outdoor_environment', 'belgiumChieverseAirportTrain_10min']],
   'processParam': {'refine_param': {'removeDuplication': {'flag': False},
     'staticFrequency': {'flag': False, 'frequency': None}},
    'outlier_param': {'certainErrorToNaN': {'flag': False},
     'unCertainErrorToNaN': {'flag': False, 'param': {}}},
    'imputation_param': {'flag': False,
     'imputation_method': [],
     'totalNonNanRatio': 80}},
   'integration_freq_sec': 600,
   'cleanParam': 'NoClean',
   'DataSaveMode': 'CSV'},
  'featureList': ['RH_1',
   'RH_2',
   'RH_3',
   'RH_4',
   'RH_5',
   'RH_6',
   'RH_7',
   'RH_8',
   'RH_9',
   'T1',
   'T2',
   'T3',
   'T4',
   'T5',
   'T6',
   'T7',
   'T8',
   'T9',
  